In [1]:
import os
from dotenv import load_dotenv
from linkedin_api import Linkedin

In [2]:
# Load environment variables from .env file
load_dotenv()

# Get Linkedin credentials from environment variables
linkedin_username = os.getenv('LINKEDIN_USERNAME')
linkedin_password = os.getenv('LINKEDIN_PASSWORD')

# Authenticate using the credentials from the .env file
api = Linkedin(linkedin_username, linkedin_password)

In [3]:
profile = input("Profile: ")

# GET a profile
profile_data = api.get_profile(profile)
contact_info = api.get_profile_contact_info(profile)

In [4]:
# Helper function to format dates
def format_date(date):
    return f"{date.get('month', '')}/{date.get('year', '')}"

In [5]:
# Create a resume
def create_resume(profile_data, contact_info):
    # Personal Details
    full_name = f"{profile_data.get('firstName')} {profile_data.get('lastName')}"
    headline = profile_data.get('headline', '')
    location = profile_data.get('locationName', 'Unknown')
    summary = profile_data.get('summary', 'No summary available')

    # Contact Information
    email = contact_info.get('email_address', 'Not provided')
    github = next((site.get('url') for site in contact_info.get('websites', []) if site['label'] == 'PORTFOLIO'), 'Not provided')
    portfolio = contact_info.get('websites', [])
    
    # Experience
    experience_entries = profile_data.get('experience', [])
    experience_str = ""
    for exp in experience_entries:
        company = exp.get('companyName', 'N/A')
        title = exp.get('title', 'N/A')
        location = exp.get('locationName', 'N/A')
        start_date = format_date(exp.get('timePeriod', {}).get('startDate', {}))
        end_date = format_date(exp.get('timePeriod', {}).get('endDate', {})) or 'Present'
        description = exp.get('description', 'No description available')
        experience_str += f"{title} at {company} ({start_date} - {end_date})\n"
        experience_str += f"Location: {location}\n"
        experience_str += f"Description: {description}\n\n"

    # Education
    education_entries = profile_data.get('education', [])
    education_str = ""
    for edu in education_entries:
        school = edu.get('schoolName', 'Unknown School')
        field_of_study = edu.get('fieldOfStudy', 'N/A')
        degree = edu.get('degreeName', '')
        start_date = format_date(edu.get('timePeriod', {}).get('startDate', {}))
        end_date = format_date(edu.get('timePeriod', {}).get('endDate', {}))
        education_str += f"{school} - {degree} ({start_date} - {end_date})\n"
        education_str += f"Field of Study: {field_of_study}\n\n"

    # Skills
    skills = profile_data.get('skills', [])
    skills_str = ", ".join([skill.get('name') for skill in skills])

    # Certifications
    certifications = profile_data.get('certifications', [])
    cert_str = ""
    for cert in certifications:
        cert_name = cert.get('name', 'Unknown Certification')
        authority = cert.get('authority', 'Unknown Authority')
        start_date = format_date(cert.get('timePeriod', {}).get('startDate', {}))
        cert_str += f"{cert_name} by {authority} ({start_date})\n"

    # Languages
    languages = profile_data.get('languages', [])
    languages_str = ", ".join([f"{lang.get('name')} ({lang.get('proficiency')})" for lang in languages])

    # Format resume
    resume = f"""
    {full_name}
    {headline}
    {location}

    Contact Information:
    Email: {email}
    GitHub: {github}
    Portfolio: {portfolio}

    Summary:
    {summary}

    Experience:
    {experience_str}

    Education:
    {education_str}

    Skills:
    {skills_str}

    Certifications:
    {cert_str}

    Languages:
    {languages_str}
    """
    
    return resume



In [8]:
def save_resume_to_file(profile_data, contact_info):
    # Get full name and create filename without spaces and lowercase
    full_name = f"{profile_data.get('firstName', '')} {profile_data.get('lastName', '')}"
    filename = full_name.replace(" ", "").lower() + '_resume.txt'  # Remove spaces and make lowercase

    # Create the resume
    resume = create_resume(profile_data, contact_info)

    # Save to file
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(resume)
    
    print(f"Resume saved to {filename}")



In [9]:
# Create and save the resume
save_resume_to_file(profile_data, contact_info)

Resume saved to pedrolustosa_resume.txt
